In [5]:
import argparse
import json
import os
import threading
import time
from random import random
import numpy as np
import requests
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
import numpy_encoder
import yaml
from sklearn.model_selection import train_test_split
import pandas as pd
import sys
import pickle

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# setup
import pycaret
import missingno

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, log_loss
from sklearn import preprocessing
import umap
import umap.plot
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')

# Loading Dataset

In [3]:
import pathlib
current_dir = pathlib.Path.cwd()
parent_dir = current_dir.parent
result_dir = pathlib.Path.joinpath(current_dir, 'result')
pathlib.Path.mkdir(result_dir, mode=0o777, parents=True, exist_ok=True)

In [6]:
with open('./{}'.format("data_config.json"), encoding='UTF8') as file:
    cfg = json.load(file)

drop_cols = cfg['drop_cols']
selected_cols = cfg['selected_cols']
target_col = cfg['target_col']

icu_units = list(cfg['icu_units'].values())
train_data_df = pd.DataFrame()
for icu in icu_units[:-1]:
    icu_df = pd.read_feather(parent_dir.joinpath('data', 'eicu', f'{icu}.feather'))
    train_data_df = pd.concat([train_data_df, icu_df], axis=0).reset_index(drop=True)
    
valid_data_df = pd.read_feather(parent_dir.joinpath('data', 'eicu', f'{icu_units[-1]}.feather'))

X_data, y_data = train_data_df[selected_cols], train_data_df['death']
X_valid, y_valid = valid_data_df[selected_cols], valid_data_df['death']

In [ ]:
train_data = train_data.dropna(axis=1)
len(train_data.columns)

In [7]:
train_data_df = train_data_df[selected_cols + ['death']]
valid_data_df = valid_data_df[selected_cols + ['death']]

In [8]:
X_data, y_data = train_data_df[selected_cols], train_data_df['death']
X_valid, y_valid = valid_data_df[selected_cols], valid_data_df['death']

In [9]:
from fastai import *
from fastai.tabular.all import * 

In [11]:
from pycaret.classification import *
clf1 = setup(data = train_data_df, 
             target = 'death',
             preprocess = False,
             numeric_features = selected_cols,
             silent=True)

,Description,Value
0,session_id,5375
1,Target,death
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(153458, 27)"
5,Missing Values,False
6,Numeric Features,26
7,Categorical Features,0
8,Transformed Train Set,"(107420, 26)"
9,Transformed Test Set,"(46038, 26)"


In [12]:
mlp = create_model('mlp')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9770,0.9919,0.8770,0.8471,0.8618,0.8492,0.8494
1,0.9779,0.9919,0.7995,0.9201,0.8556,0.8437,0.8461
2,0.9774,0.9915,0.8428,0.8757,0.8590,0.8467,0.8469
3,0.9771,0.9919,0.8554,0.8632,0.8593,0.8468,0.8468
4,0.9805,0.9910,0.8519,0.9034,0.8769,0.8663,0.8667
5,0.9763,0.9916,0.8440,0.8626,0.8532,0.8403,0.8403
6,0.9769,0.9911,0.7952,0.9113,0.8493,0.8369,0.8392
7,0.9799,0.9912,0.8214,0.9245,0.8699,0.8590,0.8608
8,0.9770,0.9928,0.8043,0.9041,0.8513,0.8389,0.8406


In [ ]:
top5 = compare_models(sort='Accuracy',n_select = 5,
                      exclude = ['knn', 'svm','ridge','nb','dummy','qda']#,'xgboost']
                     )

In [13]:
save_model(mlp,'mlp_model_11Nov2020')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=False, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=['age', 'albumin',
                                                           'bicarbonate',
                                                           'bilirubin', 'bun',
                                                           'creatinine', 'fio2',
                                                           'gcs', 'glucose',
                                                           'hct', 'heartrate',
                                                           'hospitaladmitoffset',
                                                           'intubated', 'ph',
                                                        

In [ ]:
new_prediction = predict_model(mlp, data=valid_data_df)

In [ ]:
new_prediction

In [14]:
evaluate_model(mlp)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
from pycaret.utils import check_metric
check_metric(new_prediction['death'], new_prediction['Label'], metric = 'MAE')

In [ ]:
mlp.get_params()

In [ ]:
print(mlp)